# 📊 Vale do São Francisco: Análise da Produção de Frutas de Alto Valor (2013-2024)

---

**Autor:** Everton Santos de Oliveira | [eso.datalab@gmail.com](mailto:eso.datalab@gmail.com)

[![LinkedIn](https://img.shields.io/badge/LinkedIn-0077B5?logo=linkedin&logoColor=white)](https://www.linkedin.com/in/evert0n-sant0s/) [![GitHub](https://img.shields.io/badge/GitHub-181717?logo=github&logoColor=white)](https://github.com/everton754) [![Medium](https://img.shields.io/badge/Medium-12100E?logo=medium&logoColor=white)](https://medium.com/@eso.datalab) [![Portfolio](https://img.shields.io/badge/Portfolio-FF5722?logo=google-chrome&logoColor=white)](https://kenzie-portfolio-everton754.vercel.app/)

**Projeto:** Portfólio de Data Science — Análise Exploratória & Insights Estratégicos

---

## 🎯 **Executive Summary**

### **Contexto de Negócio**

O **Vale do São Francisco** (Pernambuco/Bahia) é o principal polo de fruticultura irrigada do Brasil, responsável por:
- **95%** da produção nacional de **uva de mesa** 🍇
- **85%** das exportações brasileiras de **manga** 🥭
- Geração de **R$ 2+ bilhões** anuais em valor de produção
- **500 mil empregos** diretos e indiretos na região

Este projeto analisa **12 anos** de dados oficiais do **IBGE (Produção Agrícola Municipal - PAM)** integrados com **Censo Agropecuário** para identificar:
1. **Tendências de crescimento** da produção regional (2013-2024)
2. **Perfis municipais** e concentração geográfica
3. **Fatores de produtividade** e comparação entre culturas
4. **Oportunidades estratégicas** para políticas públicas e investimentos privados

---

### **🔍 Perguntas de Pesquisa**

1. **Qual a evolução da produção de uva e manga no Vale do São Francisco (2013-2024)?**
2. **Quais municípios concentram a produção? Há diversificação geográfica ou alta concentração?**
3. **Há diferenças significativas de produtividade (rendimento) entre uva e manga?**
4. **Quais são os padrões temporais (tendências, sazonalidade, aceleração/desaceleração)?**
5. **Que correlações existem entre área plantada, produção, rendimento e valor da produção?**

---

### **📊 Principais Resultados (Prévia)**

> **Nota:** Os resultados detalhados serão apresentados ao longo da análise.

| Métrica | Valor | Insight |
| :--- | :--- | :--- |
| **🚀 Crescimento (12 anos)** | **Uva:** +252% (CAGR 12.1%) <br> **Manga:** +82% (CAGR 5.2%) | Evolução total da produção por cultura (2013-2024). |
| **🥇 Município Líder (2024)** | **Petrolina (PE) – 68.7% da produção** | Concentração da produção regional total (manga + uva) em 2024. |
| **📈 Rendimento Médio** | **Manga:** 29,357 kg/ha <br> **Uva:** 38,030 kg/ha | Produtividade média no período, com uva 29.5% superior. |
| **💰 Valor Total (2024)** | **R\$ 2.68 Bi** (vs R\$ 0.95 Bi em 2013) | Crescimento de 182% em 11 anos, impulsionado por uva. |

> 💡 **Nota:** A tabela acima é baseada nos indicadores calculados ao longo do notebook (produção, rendimento e valor) a partir do dataset integrado corrigido (48 registros).

---

### **🛠️ Tecnologias e Metodologia**

**Stack Técnico:**
- **Python** 3.11+ • **Pandas** 2.0+ • **NumPy** 1.24+
- **Matplotlib** 3.7+ • **Seaborn** 0.12+
- **Scipy** 1.10+ (testes estatísticos)
- **Scikit-learn** 1.3+ (feature importance)

**Pipeline de Análise:**
1. **Data Ingestion:** Carregamento de dados processados (IBGE PAM + Censo Agro integrado)
2. **Data Quality:** Análise de missing values, outliers e consistência
3. **EDA:** Análises univariada, bivariada e multivariada
4. **Statistical Testing:** ANOVA, Mann-Whitney, correlações
5. **Visualization:** Gráficos profissionais com paleta customizada (Agronegócio)
6. **Insights:** Interpretação de resultados + recomendações estratégicas

---

**📅 Última Atualização:** 13 de novembro de 2025  
**⏱️ Tempo Estimado de Execução:** 3-5 minutos (Google Colab)  
**📁 Dataset:** `pam_censo_agro_integrado_v2.csv` (2013–2024, 48 registros)

---


In [ ]:
# ============================================================================
# CÉLULA 1: SETUP E IMPORTS
# ============================================================================
# Descrição: Instalação de dependências e importação de bibliotecas necessárias
# Ambiente: Google Colab com Google Drive montado
# ============================================================================

# --- 1. INSTALAÇÃO DE DEPENDÊNCIAS ---
# Instalar bibliotecas específicas (se necessário)
# Remova o comentário (#) se alguma biblioteca não estiver disponível

# !pip install --quiet --upgrade pandas numpy matplotlib seaborn scipy scikit-learn
# !pip install --quiet missingno  # Para visualização de missing values (opcional)

# --- 2. IMPORTAÇÕES GERAIS ---
import warnings
warnings.filterwarnings('ignore')  # Suprimir warnings para limpeza visual

# Bibliotecas de manipulação de dados
import pandas as pd
import numpy as np

# Bibliotecas de visualização
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rcParams

# Bibliotecas estatísticas
from scipy import stats
from scipy.stats import shapiro, mannwhitneyu, ttest_ind, f_oneway, spearmanr

# Bibliotecas de Machine Learning (para feature importance)
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler

# Utilitários
import os
from datetime import datetime
from pathlib import Path

# --- 3. MONTAGEM DO GOOGLE DRIVE ---
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=False)
    print("✅ Google Drive montado com sucesso!")
    COLAB_ENV = True
except:
    print("⚠️ Não está rodando no Google Colab. Usando ambiente local.")
    COLAB_ENV = False

# --- 4. CONFIGURAÇÕES GLOBAIS ---
# Configurar display do Pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)
pd.set_option('display.width', 1000)

# Seed para reprodutibilidade
np.random.seed(42)

# --- 5. VERIFICAÇÃO DE VERSÕES ---
print("\n" + "="*80)
print("📦 VERSÕES DAS BIBLIOTECAS")
print("="*80)
print(f"Python: {pd.__version__ if hasattr(pd, '__version__') else 'N/A'}")
print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")
print(f"Matplotlib: {plt.matplotlib.__version__}")
print(f"Seaborn: {sns.__version__}")
print(f"Scipy: {stats.__version__ if hasattr(stats, '__version__') else 'N/A'}")
print(f"Scikit-learn: {StandardScaler.__module__.split('.')[0]}")
print("="*80)

# --- 6. INFORMAÇÕES DO AMBIENTE ---
print("\n" + "="*80)
print("🖥️ INFORMAÇÕES DO AMBIENTE")
print("="*80)
print(f"Ambiente: {'Google Colab' if COLAB_ENV else 'Local/Jupyter'}")
print(f"Data/Hora da Execução: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}")
print(f"Diretório Atual: {os.getcwd()}")
print("="*80)

print("\n✅ Setup concluído! Pronto para carregar os dados.")

In [ ]:
# ============================================================================
# CÉLULA 2: CONFIGURAÇÕES GLOBAIS DE VISUALIZAÇÃO
# ============================================================================
# Descrição: Define paleta de cores, estilos e parâmetros de gráficos
# Objetivo: Criar identidade visual profissional com tema Agronegócio
# ============================================================================

# --- 1. PALETA DE CORES CUSTOMIZADA (AGRONEGÓCIO) ---
CUSTOM_PALETTE = {
    'primary': '#2E7D32',      # Verde escuro (vegetação/agricultura)
    'secondary': '#FFA000',    # Laranja (sol/manga/energia)
    'accent': '#6A1B9A',       # Roxo (uva/sofisticação)
    'neutral': '#757575',      # Cinza (texto/linhas)
    'background': '#F5F5F5',   # Off-white (fundo)
    'success': '#4CAF50',      # Verde claro (positivo)
    'warning': '#FF9800',      # Laranja claro (atenção)
    'danger': '#F44336',       # Vermelho (negativo/outliers)
    'info': '#2196F3'          # Azul (informação)
}

# Paletas para diferentes tipos de gráficos
COLORS_LINE = [CUSTOM_PALETTE['accent'], CUSTOM_PALETTE['secondary']]  # Uva (roxo) e Manga (laranja)
COLORS_BAR = [CUSTOM_PALETTE['primary'], CUSTOM_PALETTE['secondary'], CUSTOM_PALETTE['accent']]
COLORS_SEQUENTIAL = sns.color_palette("YlGn", as_cmap=False)  # Verde sequencial
COLORS_DIVERGENT = sns.diverging_palette(10, 130, as_cmap=False)  # Divergente (vermelho-verde)

# --- 2. CONFIGURAÇÕES MATPLOTLIB (rcParams) ---
plt.rcParams.update({
    # Figura e resolução
    'figure.figsize': (14, 6),
    'figure.dpi': 120,              # Boa qualidade para notebooks
    'savefig.dpi': 300,             # Alta qualidade para exportação
    'savefig.bbox': 'tight',
    'savefig.pad_inches': 0.1,

    # Fontes
    'font.family': 'sans-serif',
    'font.sans-serif': ['DejaVu Sans', 'Arial', 'Helvetica', 'Liberation Sans'],
    'font.size': 11,

    # Títulos e labels
    'axes.titlesize': 14,
    'axes.titleweight': 'bold',
    'axes.labelsize': 11,
    'axes.labelweight': 'normal',
    'axes.labelcolor': CUSTOM_PALETTE['neutral'],

    # Ticks
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'xtick.color': CUSTOM_PALETTE['neutral'],
    'ytick.color': CUSTOM_PALETTE['neutral'],

    # Legendas
    'legend.fontsize': 10,
    'legend.frameon': True,
    'legend.framealpha': 0.9,
    'legend.edgecolor': '#CCCCCC',
    'legend.fancybox': True,

    # Grid
    'axes.grid': True,
    'grid.alpha': 0.3,
    'grid.linestyle': '--',
    'grid.linewidth': 0.5,
    'grid.color': '#CCCCCC',

    # Spines (bordas)
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.spines.left': True,
    'axes.spines.bottom': True,
    'axes.edgecolor': CUSTOM_PALETTE['neutral'],
    'axes.linewidth': 1.0,

    # Cores de fundo
    'axes.facecolor': 'white',
    'figure.facecolor': 'white',
    'savefig.facecolor': 'white',
})

# --- 3. CONFIGURAÇÕES SEABORN ---
sns.set_theme(
    style='whitegrid',
    palette='muted',
    context='notebook',  # Opções: paper, notebook, talk, poster
    rc={
        'axes.spines.top': False,
        'axes.spines.right': False,
    }
)

# --- 4. FUNÇÕES AUXILIARES PARA GRÁFICOS ---

def format_large_numbers(value, pos=None):
    """
    Formata números grandes para escala legível (milhares, milhões).
    Uso: ax.yaxis.set_major_formatter(FuncFormatter(format_large_numbers))
    """
    if value >= 1_000_000:
        return f'{value/1_000_000:.1f}M'
    elif value >= 1_000:
        return f'{value/1_000:.0f}K'
    else:
        return f'{value:.0f}'

def add_value_labels(ax, spacing=5, fontsize=9):
    """
    Adiciona rótulos de valor no topo de barras verticais ou horizontais.

    Parâmetros:
    - ax: objeto de eixo do matplotlib
    - spacing: espaço entre barra e texto (pixels)
    - fontsize: tamanho da fonte dos rótulos
    """
    for rect in ax.patches:
        # Obter altura/largura da barra
        if rect.get_width() > 0:  # Barra horizontal
            value = rect.get_width()
            y_pos = rect.get_y() + rect.get_height() / 2
            x_pos = value + spacing
            ax.text(x_pos, y_pos, f'{value:,.0f}',
                    ha='left', va='center', fontsize=fontsize)
        elif rect.get_height() > 0:  # Barra vertical
            value = rect.get_height()
            x_pos = rect.get_x() + rect.get_width() / 2
            y_pos = value + spacing
            ax.text(x_pos, y_pos, f'{value:,.0f}',
                    ha='center', va='bottom', fontsize=fontsize)

def create_subtitle(fig, text, y_position=0.96):
    """
    Adiciona um subtítulo centralizado à figura.

    Parâmetros:
    - fig: objeto de figura do matplotlib
    - text: texto do subtítulo
    - y_position: posição vertical (0-1)
    """
    fig.text(0.5, y_position, text,
             ha='center', va='top',
             fontsize=12, color=CUSTOM_PALETTE['neutral'],
             style='italic')

# --- 5. TESTAR PALETA ---
print("="*80)
print("🎨 PALETA DE CORES AGRONEGÓCIO")
print("="*80)

# Criar gráfico de demonstração da paleta
fig, ax = plt.subplots(figsize=(12, 3))
colors_list = list(CUSTOM_PALETTE.values())
color_names = list(CUSTOM_PALETTE.keys())

for i, (name, color) in enumerate(CUSTOM_PALETTE.items()):
    ax.barh(i, 1, color=color, edgecolor='black', linewidth=0.5)
    ax.text(0.5, i, f'{name.upper()} ({color})',
            ha='center', va='center', fontsize=11,
            color='white' if i < 3 else 'black', fontweight='bold')

ax.set_xlim(0, 1)
ax.set_ylim(-0.5, len(CUSTOM_PALETTE) - 0.5)
ax.axis('off')
ax.set_title('Paleta de Cores Customizada — Tema Agronegócio',
             fontsize=14, fontweight='bold', pad=15)

plt.tight_layout()
plt.show()

print("\n✅ Configurações de visualização aplicadas com sucesso!")
print(f"   - Resolução padrão: {plt.rcParams['figure.figsize']}")
print(f"   - DPI (notebook): {plt.rcParams['figure.dpi']}")
print(f"   - DPI (exportação): {plt.rcParams['savefig.dpi']}")
print(f"   - Paleta principal: {len(CUSTOM_PALETTE)} cores")
print("="*80)

## 📂 CÉLULA 3 – Carregamento do dataset **corrigido** (sem merge cartesiano)

Nesta etapa, o objetivo é carregar o novo dataset integrado `pam_censo_agro_integrado_v2.csv`, gerado no notebook `02_data_cleaning_FIXED.ipynb` com **48 registros** (2 municípios × 12 anos × 2 produtos).  

Após carregar o CSV, vamos:
- Padronizar os nomes das colunas para `snake_case` e minúsculas.  
- Reconstruir as colunas utilizadas ao longo da EDA original:  
  - `quantidade_produzida_ton` (a partir de `quantidade_produzida_t`),  
  - `preco_medio_anual_r$_kg` (a partir de `preco_medio_r$_kg`),  
  - `valor_producao_r$` ≈ `quantidade_produzida_ton × 1000 × preco_medio_anual_r$_kg` (aproximação em R$ com base em preço médio anual).  

Com isso, todo o restante do notebook (`df_original`, `df_agregado`, análises temporais, geográficas, correlações e testes estatísticos) continua funcionando sem alterações estruturais.  

In [ ]:
# ============================================================================
# CÉLULA 3: CARREGAMENTO DE DADOS (VERSÃO CORRIGIDA)
# ============================================================================
# Descrição: Carrega o dataset integrado corrigido (sem merge cartesiano)
# Arquivo: pam_censo_agro_integrado_v2.csv (2013-2024, 48 registros)
# ============================================================================

# --- 1. DEFINIR CAMINHO DO ARQUIVO ---

DATA_PATH = "/content/drive/MyDrive/projeto_produtividade/data/processed/pam_censo_agro_integrado_v2.csv"

# Verificar se arquivo existe
if not os.path.exists(DATA_PATH):
    print("❌ ERRO: Arquivo não encontrado no caminho especificado!")
    print(f" Caminho: {DATA_PATH}")
    print("\n💡 Verifique:")
    print(" 1. O Google Drive está montado corretamente?")
    print(" 2. O caminho do arquivo está correto?")
    print(" 3. O arquivo existe na pasta?")
    raise FileNotFoundError(f"Arquivo não encontrado: {DATA_PATH}")
else:
    print(f"✅ Arquivo encontrado: {DATA_PATH}")

# --- 2. CARREGAR DATASET ---

print("\n" + "="*80)
print("📊 CARREGANDO DATASET (VERSÃO CORRIGIDA)")
print("="*80)

try:
    # Importante: usar o mesmo encoding da etapa de cleaning
    df = pd.read_csv(DATA_PATH, encoding="utf-8-sig")
    print("✅ Dataset carregado com sucesso!")
except Exception as e:
    print(f"❌ ERRO ao carregar arquivo: {e}")
    raise

# --- 3. PADRONIZAR NOMES DE COLUNAS E TIPOS ---

# Garantir nomes em minúsculas e sem espaços
df.columns = [col.strip().lower() for col in df.columns]

print("\n📋 Colunas após padronização de nomes:")
print(df.columns.tolist())

# Dataset atual esperado (a partir do merge corrigido):
# ['municipio', 'ano', 'produto',
#  'area_colhida_ha', 'quantidade_produzida_t',
#  'rendimento_medio_kg_ha',
#  'preco_mediano_r$_kg', 'preco_medio_r$_kg',
#  'preco_std_r$_kg', 'num_observacoes_preco']

# 3.1 Criar coluna quantidade_produzida_ton (mesma unidade em tons)
if "quantidade_produzida_t" in df.columns and "quantidade_produzida_ton" not in df.columns:
    df["quantidade_produzida_ton"] = df["quantidade_produzida_t"].astype(float)

# 3.2 Criar coluna preco_medio_anual_r$_kg a partir do preco_medio_r$_kg
if "preco_medio_r$_kg" in df.columns and "preco_medio_anual_r$_kg" not in df.columns:
    df["preco_medio_anual_r$_kg"] = df["preco_medio_r$_kg"].astype(float)

# 3.3 Criar coluna valor_producao_r$ ≈ quantidade * preço médio (ton → kg)
if "valor_producao_r$" not in df.columns:
    df["valor_producao_r$"] = (
        df["quantidade_produzida_ton"].astype(float) * 1000.0 * df["preco_medio_anual_r$_kg"].astype(float)
    )

# Assegurar tipos principais
df["ano"] = df["ano"].astype(int)
df["municipio"] = df["municipio"].astype(str)
df["produto"] = df["produto"].astype(str)

# --- 4. INFORMAÇÕES BÁSICAS DO DATASET ---

print("\n" + "="*80)
print("📋 INFORMAÇÕES GERAIS DO DATASET (VERSÃO CORRIGIDA)")
print("="*80)

print("\n🔢 Dimensões:")
print(f" - Linhas (observações): {df.shape[0]:,}")
print(f" - Colunas (variáveis): {df.shape[1]}")
print(f" - Tamanho em memória: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print("\n📅 Período dos Dados:")
if "ano" in df.columns or "Ano" in df.columns:
    ano_col = "ano" if "ano" in df.columns else "Ano"
    print(f" - Ano mínimo: {df[ano_col].min()}")
    print(f" - Ano máximo: {df[ano_col].max()}")
    print(f" - Período total: {df[ano_col].max() - df[ano_col].min() + 1} anos")
else:
    print(" - Coluna 'ano' não identificada automaticamente")

print("\n📂 Colunas do Dataset:")
for i, col in enumerate(df.columns, 1):
    print(f" {i:2d}. {col} ({df[col].dtype})")

print("="*80)

# --- 5. PRIMEIRAS E ÚLTIMAS LINHAS ---

print("\n" + "="*80)
print("👁️ PRIMEIRAS 5 LINHAS DO DATASET")
print("="*80)
display(df.head())

print("\n" + "="*80)
print("👁️ ÚLTIMAS 5 LINHAS DO DATASET")
print("="*80)
display(df.tail())

# --- 6. INFORMAÇÕES DETALHADAS E ESTATÍSTICAS BÁSICAS ---

print("\n" + "="*80)
print("ℹ️ INFORMAÇÕES DETALHADAS (TIPOS E MISSING VALUES)")
print("="*80)
df.info()

print("\n" + "="*80)
print("📊 ESTATÍSTICAS DESCRITIVAS BÁSICAS (VARIÁVEIS NUMÉRICAS)")
print("="*80)
display(df.describe().T)

# --- 7. VERIFICAÇÃO DE COLUNAS CRÍTICAS PARA A EDA ---

print("\n" + "="*80)
print("✅ VERIFICAÇÃO DE COLUNAS CRÍTICAS PARA A EDA")
print("="*80)

EXPECTED_COLUMNS = [
    "municipio",
    "ano",
    "produto",
    "area_colhida_ha",
    "quantidade_produzida_ton",
    "rendimento_medio_kg_ha",
    "preco_medio_anual_r$_kg",
    "valor_producao_r$",
]

missing_cols = [col for col in EXPECTED_COLUMNS if col not in df.columns]
if missing_cols:
    print(f"⚠️ Colunas esperadas não encontradas: {missing_cols}")
    print("💡 Verifique os nomes das colunas no dataset e ajuste a análise se necessário.")
else:
    print("✅ Todas as colunas críticas estão presentes!")

print("\n✅ Dados carregados e prontos para análise exploratória com o dataset corrigido!")
print("="*80)

def calcular_cagr(valor_inicial: float, valor_final: float, n_periodos: int) -> float:
    """
    Calcula o CAGR (taxa média de crescimento anual).
    """
    if valor_inicial <= 0 or n_periodos <= 0:
        return np.nan
    return (valor_final / valor_inicial) ** (1 / n_periodos) - 1

## 📌 Snapshot Executivo – Dados Corrigidos (48 registros)

Antes de avançar para a análise exploratória detalhada, é importante validar – em nível executivo – se o dataset **corrigido** faz sentido em termos de magnitude e trajetória ao longo do tempo.  

Nesta seção vamos calcular alguns **KPIs chave** usando o dataset `pam_censo_agro_integrado_v2.csv` já integrado IBGE + CEPEA:

- Produção total (toneladas) em 2013 vs 2024, por município e produto.  
- CAGR (taxa média de crescimento anual) da produção e do valor de produção no período 2013–2024.  
- Ticket médio aproximado (R$/kg) por município e produto no início e no fim da série.  

Esses números servem tanto como **checagem de consistência** quanto como base para o storytelling da EDA nas próximas seções.  


In [ ]:
# ============================================================================
# CÉLULA 4: SNAPSHOT EXECUTIVO – KPIs CHAVE (DADOS CORRIGIDOS)
# ============================================================================


# 1. Produção total (ton) em 2013 vs 2024 por município e produto
anos_extremos = [df["ano"].min(), df["ano"].max()]

df_extremos = (
    df[df["ano"].isin(anos_extremos)]
    .groupby(["ano", "municipio", "produto"], as_index=False)
    .agg(
        {
            "quantidade_produzida_ton": "sum",
            "valor_producao_r$": "sum",
            "preco_medio_anual_r$_kg": "mean",
        }
    )
)

# Renomear colunas agregadas para nomes mais legíveis
df_extremos = df_extremos.rename(columns={
    "quantidade_produzida_ton": "producao_total_ton",
    "valor_producao_r$": "valor_producao_total_r$",
    "preco_medio_anual_r$_kg": "preco_medio_r$_kg",
})

print("="*80)
print("📌 PRODUÇÃO E VALOR – ANOS INICIAIS VS FINAIS (por município e produto)")
print("="*80)
display(df_extremos.sort_values(["ano", "municipio", "produto"]))


# 2. CAGR de produção e valor de produção 2013–2024 (por município e produto)
ano_min = df["ano"].min()
ano_max = df["ano"].max()
n_periodos = ano_max - ano_min  # ex.: 2024 - 2013 = 11 -> 11 intervalos

kpis_cagr = []

for (municipio, produto), grupo in df.groupby(["municipio", "produto"]):
    grupo_sorted = grupo.sort_values("ano")

    prod_ini = grupo_sorted.loc[grupo_sorted["ano"] == ano_min, "quantidade_produzida_ton"].sum()
    prod_fim = grupo_sorted.loc[grupo_sorted["ano"] == ano_max, "quantidade_produzida_ton"].sum()

    val_ini = grupo_sorted.loc[grupo_sorted["ano"] == ano_min, "valor_producao_r$"].sum()
    val_fim = grupo_sorted.loc[grupo_sorted["ano"] == ano_max, "valor_producao_r$"].sum()

    cagr_prod = calcular_cagr(prod_ini, prod_fim, n_periodos)
    cagr_valor = calcular_cagr(val_ini, val_fim, n_periodos)

    kpis_cagr.append(
        {
            "municipio": municipio,
            "produto": produto,
            "producao_inicial_ton": prod_ini,
            "producao_final_ton": prod_fim,
            "cagr_producao_%": cagr_prod * 100 if not np.isnan(cagr_prod) else np.nan,
            "valor_inicial_r$": val_ini,
            "valor_final_r$": val_fim,
            "cagr_valor_%": cagr_valor * 100 if not np.isnan(cagr_valor) else np.nan,
        }
    )

df_kpis_cagr = pd.DataFrame(kpis_cagr)

print("\n" + "="*80)
print("📌 CAGR PRODUÇÃO E VALOR – 2013–2024 (por município e produto)")
print("="*80)
display(df_kpis_cagr.sort_values(["municipio", "produto"]))


# 3. Ticket médio (R$/kg) no início vs fim da série
df_ticket = (
    df[df["ano"].isin(anos_extremos)]
    .groupby(["ano", "municipio", "produto"], as_index=False) # Incluir 'ano' na agregação
    .agg({"preco_medio_anual_r$_kg": "mean"})
    .pivot_table(
        index=["municipio", "produto"],
        columns="ano",
        values="preco_medio_anual_r$_kg"
    )
)

# Renomear colunas de anos para algo mais legível
df_ticket.columns = [f"preco_medio_{int(col)}_r$_kg" for col in df_ticket.columns]
df_ticket = df_ticket.reset_index()

print("\n" + "="*80)
print("📌 TICKET MÉDIO R$/kg – INÍCIO VS FIM DA SÉRIE")
print("="*80)
display(df_ticket)

## 📈 Visão Executiva – Produção, Valor e Preço (2013–2024)

Com o dataset corrigido e os KPIs calculados, vamos construir três gráficos executivos para apoiar a tomada de decisão:

1. **Produção total (ton)** por ano, município e produto – para entender quem mais cresce em volume.  
2. **Valor da produção (R$ milhões)** por ano, município e produto – capturando o impacto econômico da fruticultura.  
3. **Preço médio anual (R$/kg)** por ano, município e produto – destacando a evolução do “ticket médio” de manga e uva.  

Esses gráficos funcionam como um “painel de bordo” do Vale do São Francisco, conectando área, produtividade, preço e valor de forma visual e direta para gestores públicos e investidores.  


In [ ]:
# ============================================================================
# CÉLULA 5: GRÁFICOS EXECUTIVOS – PRODUÇÃO, VALOR E PREÇO
# ============================================================================

import matplotlib.pyplot as plt
import seaborn as sns

# Copiar dataframe base para evitar modificações acidentais
df_plot = df.copy()

# Criar coluna de valor em milhões de R$
df_plot["valor_producao_milhoes_r$"] = df_plot["valor_producao_r$"] / 1e6

sns.set_style("whitegrid")
sns.set_palette(COLORS_LINE)  # usa a mesma paleta customizada do restante da EDA

# --------------------------------------------------------------------------
# 1. Produção total (ton) por ano, município e produto
# --------------------------------------------------------------------------

g1 = sns.relplot(
    data=df_plot,
    x="ano",
    y="quantidade_produzida_ton",
    hue="produto",
    col="municipio",
    kind="line",
    marker="o",
    height=4,
    aspect=1.4,
)

g1.set_titles("{col_name}")
g1.set_axis_labels("Ano", "Produção (ton)")
g1._legend.set_title("Produto")

for ax in g1.axes.flat:
    ax.ticklabel_format(axis="y", style="plain", useOffset=False)
    ax.set_ylim(bottom=0)

plt.subplots_adjust(top=0.8)
g1.fig.suptitle("Evolução da Produção de Manga e Uva (2013–2024)", fontsize=14, y=1.05)
plt.show()

# --------------------------------------------------------------------------
# 2. Valor da produção (R$ milhões) por ano, município e produto
# --------------------------------------------------------------------------

g2 = sns.relplot(
    data=df_plot,
    x="ano",
    y="valor_producao_milhoes_r$",
    hue="produto",
    col="municipio",
    kind="line",
    marker="o",
    height=4,
    aspect=1.4,
)

g2.set_titles("{col_name}")
g2.set_axis_labels("Ano", "Valor da produção (R$ milhões)")
g2._legend.set_title("Produto")

for ax in g2.axes.flat:
    ax.ticklabel_format(axis="y", style="plain", useOffset=False)
    ax.set_ylim(bottom=0)

plt.subplots_adjust(top=0.8)
g2.fig.suptitle("Evolução do Valor da Produção (2013–2024)", fontsize=14, y=1.05)
plt.show()

# --------------------------------------------------------------------------
# 3. Preço médio anual (R$/kg) por ano, município e produto
# --------------------------------------------------------------------------

g3 = sns.relplot(
    data=df_plot,
    x="ano",
    y="preco_medio_anual_r$_kg",
    hue="produto",
    col="municipio",
    kind="line",
    marker="o",
    height=4,
    aspect=1.4,
)

g3.set_titles("{col_name}")
g3.set_axis_labels("Ano", "Preço médio anual (R$/kg)")
g3._legend.set_title("Produto")

for ax in g3.axes.flat:
    ax.ticklabel_format(axis="y", style="plain", useOffset=False)
    ax.set_ylim(bottom=0)

plt.subplots_adjust(top=0.8)
g3.fig.suptitle("Evolução do Preço Médio Anual (R$/kg)", fontsize=14, y=1.05)
plt.show()

## 🧠 Principais Insights Executivos (2013–2024)

1. **Preço da uva disparou, manga cresceu de forma mais suave**  
   - O preço médio da uva praticamente **mais do que dobrou** entre 2013 e 2024, saindo da faixa de ~4,4 R\$/kg para algo próximo de 10,8–11 R\$/kg em ambos os municípios.  
   - Já a manga apresenta um crescimento mais moderado de ticket, saindo de ~1,3 R\$/kg para algo próximo de 2,9–3 R$/kg no mesmo período.  

2. **Petrolina consolida liderança em valor de produção de uva**  
   - Em 2013, Petrolina já produzia mais uva do que Juazeiro, mas em 2024 a diferença se torna expressiva, com algo em torno de 610 mil toneladas contra algo perto de 60 mil toneladas em Juazeiro.  
   - Com preços médios similares entre os municípios, essa diferença de volume faz o valor da produção de uva em Petrolina superar com folga 6 bilhões de reais em 2024, contra algo em torno de 650 milhões em Juazeiro.  

3. **Manga cresce em ambos os municípios, mas Juazeiro assume protagonismo em volume**  
   - A produção de manga cresce de ~210 mil para ~418 mil toneladas em Juazeiro, enquanto em Petrolina evolui de ~173 mil para ~282 mil toneladas entre 2013 e 2024.  
   - Isso sugere uma especialização relativa: Juazeiro mais forte em manga, Petrolina mais forte em uva, ambos beneficiados pela mesma tendência de aumento de preços ao longo da década.  

4. **CAGR reforça a atratividade econômica da uva**  
   - As taxas de crescimento anual composto (CAGR) do valor de produção de uva são superiores a 20% ao ano em Petrolina e acima de 16% ao ano em Juazeiro, combinando ganho de volume com ganho de preço.  
   - Para manga, o CAGR de valor fica em torno de 12–14% ao ano, ainda muito atrativo, mas abaixo do “boom” observado na cadeia da uva.  

Em conjunto, esses resultados apontam o **Vale do São Francisco** como um polo em que a uva de alta qualidade se torna o principal motor de valor em Petrolina, enquanto a manga consolida Juazeiro como importante produtor em volume, o que pode orientar decisões de investimento em tecnologia, logística e contratos de longo prazo por produto e município.  


In [ ]:
# ============================================================================
# CÉLULA 6: KPIs PARA O EXECUTIVE SUMMARY
# ============================================================================

from IPython.display import display, Markdown
import numpy as np

ano_min = df["ano"].min()
ano_max = df["ano"].max()
n_periodos = ano_max - ano_min  # ex.: 2024 - 2013 = 11

# 1) CAGR do VALOR de produção por produto (Uva x Manga)
resumo_produtos = []

for produto, grupo in df.groupby("produto"):
    valor_ini = grupo.loc[grupo["ano"] == ano_min, "valor_producao_r$"].sum()
    valor_fim = grupo.loc[grupo["ano"] == ano_max, "valor_producao_r$"].sum()
    cagr_valor = calcular_cagr(valor_ini, valor_fim, n_periodos) * 100

    resumo_produtos.append(
        {
            "produto": produto,
            "valor_inicial_r$": valor_ini,
            "valor_final_r$": valor_fim,
            "cagr_valor_%": cagr_valor,
        }
    )

df_resumo_produtos = pd.DataFrame(resumo_produtos).sort_values("produto")

# Extrair números formatados
cagr_uva = float(df_resumo_produtos.loc[df_resumo_produtos["produto"] == "Uva", "cagr_valor_%"])
cagr_manga = float(df_resumo_produtos.loc[df_resumo_produtos["produto"] == "Manga", "cagr_valor_%"])

# 2) Município líder em produção total em 2024 (% da produção regional)
df_2024 = df[df["ano"] == ano_max].groupby("municipio", as_index=False)["quantidade_produzida_ton"].sum()
df_2024["share_%"] = df_2024["quantidade_produzida_ton"] / df_2024["quantidade_produzida_ton"].sum() * 100

municipio_lider = df_2024.sort_values("quantidade_produzida_ton", ascending=False).iloc[0]["municipio"]
share_lider = df_2024.sort_values("quantidade_produzida_ton", ascending=False).iloc[0]["share_%"]

# 3) Rendimento médio (kg/ha) de Manga vs Uva no período
df_rendimento = (
    df.groupby("produto", as_index=False)["rendimento_medio_kg_ha"]
    .mean()
)

rend_manga = float(df_rendimento.loc[df_rendimento["produto"] == "Manga", "rendimento_medio_kg_ha"])
rend_uva = float(df_rendimento.loc[df_rendimento["produto"] == "Uva", "rendimento_medio_kg_ha"])

# 4) Valor total de produção em 2013 vs 2024 (todas as culturas e municípios)
valor_2013 = df[df["ano"] == ano_min]["valor_producao_r$"].sum()
valor_2024 = df[df["ano"] == ano_max]["valor_producao_r$"].sum()
multiplo_valor = valor_2024 / valor_2013
cagr_valor_total = calcular_cagr(valor_2013, valor_2024, n_periodos) * 100

# Conversões para bilhões
valor_2013_bi = valor_2013 / 1e9
valor_2024_bi = valor_2024 / 1e9

# ---------------------------------------------------------------------------
# Montar tabela em Markdown para colar no Executive Summary
# ---------------------------------------------------------------------------

tabela_md = f"""
| Métrica | Valor | Insight |
|--------|-------|---------|
| 680 Crescimento (12 anos) | Uva: {cagr_uva:.1f}% a.a. • Manga: {cagr_manga:.1f}% a.a. | CAGR do **valor de produção** por cultura entre {ano_min} e {ano_max}. |
| 947 Município Líder (2024) | {municipio_lider} – {share_lider:.1f}% da produção | Concentração da produção regional total (manga + uva) em {ano_max}. |
| 4C8 Rendimento Manga vs Uva | Manga: {rend_manga:,.0f} kg/ha • Uva: {rend_uva:,.0f} kg/ha | Produtividade média no período, evidenciando diferenças entre culturas. |
| 4B0 Valor Total de Produção (2024) | R$ {valor_2024_bi:,.2f} bi (vs R$ {valor_2013_bi:,.2f} bi em {ano_min}) | Crescimento de {multiplo_valor:,.1f}× em {n_periodos} anos (~{cagr_valor_total:.1f}% a.a.). |
"""

display(df_resumo_produtos)
display(df_2024)
display(df_rendimento)

print("\nTabela em Markdown para colar no Executive Summary:\n")
display(Markdown(tabela_md))


In [ ]:
# ============================================================================
# CÉLULA 7: DATA QUALITY ASSESSMENT
# ============================================================================
# Descrição: Análise de missing values, outliers, duplicatas e consistência
# Objetivo: Garantir qualidade e confiabilidade dos dados antes da EDA
# ============================================================================

print("="*80)
print("🔍 ANÁLISE DE QUALIDADE DOS DADOS")
print("="*80)

# --- 1. ANÁLISE DE MISSING VALUES ---
print("\n" + "="*80)
print("❓ ANÁLISE DE VALORES AUSENTES (MISSING VALUES)")
print("="*80)

missing_analysis = pd.DataFrame({
    'Coluna': df.columns,
    'Missing Count': df.isnull().sum(),
    'Missing %': (df.isnull().sum() / len(df) * 100).round(2),
    'Non-Missing': df.notnull().sum(),
    'Dtype': df.dtypes
})

missing_analysis = missing_analysis.sort_values('Missing %', ascending=False)
display(missing_analysis)

total_missing = df.isnull().sum().sum()
if total_missing == 0:
    print("\n✅ EXCELENTE! Não há valores ausentes no dataset.")
else:
    print(f"\n⚠️ Total de valores ausentes: {total_missing} ({total_missing/(df.shape[0]*df.shape[1])*100:.2f}% do dataset)")

# Visualização de missing values (se houver)
if total_missing > 0:
    fig, ax = plt.subplots(figsize=(12, 6))
    missing_pct = (df.isnull().sum() / len(df) * 100).sort_values(ascending=False)
    missing_pct = missing_pct[missing_pct > 0]

    ax.barh(missing_pct.index, missing_pct.values, color=CUSTOM_PALETTE['danger'])
    ax.set_xlabel('% de Valores Ausentes', fontweight='bold')
    ax.set_title('Percentual de Missing Values por Coluna', fontweight='bold', pad=15)
    ax.grid(axis='x', alpha=0.3)

    for i, v in enumerate(missing_pct.values):
        ax.text(v + 0.5, i, f'{v:.1f}%', va='center', fontsize=10)

    plt.tight_layout()
    plt.show()

# --- 2. ANÁLISE DE DUPLICATAS ---
print("\n" + "="*80)
print("🔄 ANÁLISE DE DUPLICATAS")
print("="*80)

duplicates_all = df.duplicated().sum()
print(f"Linhas completamente duplicadas: {duplicates_all}")

# Duplicatas por combinação município-ano-produto
duplicates_key = df.duplicated(subset=['municipio', 'ano', 'produto']).sum()
print(f"Duplicatas por (município + ano + produto): {duplicates_key}")

if duplicates_key > 0:
    print(f"\n⚠️ ATENÇÃO: Há {duplicates_key} registros duplicados na chave primária!")
    print("Mostrando alguns exemplos:")
    duplicated_rows = df[df.duplicated(subset=['municipio', 'ano', 'produto'], keep=False)]
    display(duplicated_rows.sort_values(['municipio', 'ano', 'produto']).head(10))

    print("\n💡 Possível explicação: Múltiplas observações por ano (safras, variedades, etc.)")
else:
    print("✅ Não há duplicatas na chave primária (município + ano + produto)")

# --- 3. ANÁLISE DE OUTLIERS (BOXPLOTS) ---
print("\n" + "="*80)
print("📊 ANÁLISE DE OUTLIERS (MÉTODO IQR)")
print("="*80)

# Colunas numéricas para análise de outliers
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols.remove('ano')  # Remover 'ano' da análise de outliers

# Criar subplots
n_cols = len(numeric_cols)
n_rows = (n_cols + 2) // 3  # 3 colunas por linha
fig, axes = plt.subplots(n_rows, 3, figsize=(16, n_rows * 4))
axes = axes.flatten() if n_rows > 1 else [axes]

for i, col in enumerate(numeric_cols):
    ax = axes[i]

    # Boxplot
    bp = ax.boxplot(df[col].dropna(), vert=True, patch_artist=True, widths=0.6)
    bp['boxes'][0].set_facecolor(CUSTOM_PALETTE['primary'])
    bp['boxes'][0].set_alpha(0.7)

    # Configurações
    ax.set_ylabel(col.replace('_', ' ').title(), fontweight='bold')
    ax.set_title(f'Distribuição: {col.replace("_", " ").title()}', fontweight='bold', pad=10)
    ax.grid(axis='y', alpha=0.3)

    # Estatísticas IQR
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)][col]

    # Adicionar texto com contagem de outliers
    ax.text(1.15, 0.5, f'Outliers: {len(outliers)}\n({len(outliers)/len(df)*100:.1f}%)',
            transform=ax.transAxes, va='center', fontsize=9,
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Remover subplots extras
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Análise de Outliers — Método Boxplot (IQR)',
             fontsize=16, fontweight='bold', y=1.00)
plt.tight_layout()
plt.show()

# Tabela de outliers
print("\n📋 Resumo de Outliers por Coluna:")
outlier_summary = []
for col in numeric_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]

    outlier_summary.append({
        'Coluna': col,
        'N° Outliers': len(outliers),
        '% Outliers': f"{len(outliers)/len(df)*100:.2f}%",
        'Limite Inferior': f"{lower_bound:.2f}",
        'Limite Superior': f"{upper_bound:.2f}"
    })

outlier_df = pd.DataFrame(outlier_summary)
display(outlier_df)

# --- 4. CONSISTÊNCIA DOS DADOS ---
print("\n" + "="*80)
print("✅ VERIFICAÇÃO DE CONSISTÊNCIA")
print("="*80)

# Verificar valores negativos (não devem existir em dados agrícolas)
negative_checks = {}
for col in numeric_cols:
    negative_count = (df[col] < 0).sum()
    if negative_count > 0:
        negative_checks[col] = negative_count

if negative_checks:
    print("⚠️ ATENÇÃO: Valores negativos encontrados (inconsistência):")
    for col, count in negative_checks.items():
        print(f"   - {col}: {count} valores negativos")
else:
    print("✅ Não há valores negativos (consistente com dados agrícolas)")

# Verificar zeros
zero_checks = {}
for col in numeric_cols:
    zero_count = (df[col] == 0).sum()
    if zero_count > 0:
        zero_checks[col] = zero_count

if zero_checks:
    print("\n⚠️ ATENÇÃO: Valores zero encontrados:")
    for col, count in zero_checks.items():
        print(f"   - {col}: {count} valores zero ({count/len(df)*100:.2f}%)")
else:
    print("\n✅ Não há valores zero")

# Verificar relação lógica: quantidade = área × rendimento
print("\n🔍 Verificação de Relação Lógica:")
print("   quantidade_produzida = area_colhida × rendimento_medio")

df['quantidade_calculada'] = (df['area_colhida_ha'] * df['rendimento_medio_kg_ha']) / 1000  # Converter kg para ton
df['diferenca_percentual'] = ((df['quantidade_produzida_ton'] - df['quantidade_calculada']) / df['quantidade_produzida_ton'] * 100).abs()

inconsistencias = df[df['diferenca_percentual'] > 5]  # Tolerância de 5%
print(f"   - Registros com diferença > 5%: {len(inconsistencias)} ({len(inconsistencias)/len(df)*100:.2f}%)")

if len(inconsistencias) > 0:
    print("   ⚠️ Há inconsistências na relação área × rendimento = produção")
    print("   Exemplo de registros inconsistentes:")
    display(inconsistencias[['municipio', 'ano', 'produto', 'area_colhida_ha',
                              'rendimento_medio_kg_ha', 'quantidade_produzida_ton',
                              'quantidade_calculada', 'diferenca_percentual']].head(3))
else:
    print("   ✅ Relação consistente em todos os registros")

# Remover coluna auxiliar
df.drop(['quantidade_calculada', 'diferenca_percentual'], axis=1, inplace=True)

print("\n" + "="*80)
print("✅ ANÁLISE DE QUALIDADE CONCLUÍDA!")
print("="*80)
print("\n📊 Resumo:")
print(f"   - Missing values: {total_missing} (0.00%)")
print(f"   - Duplicatas na chave primária: {duplicates_key}")
print(f"   - Valores negativos: {'Sim' if negative_checks else 'Não'}")
print(f"   - Inconsistências lógicas: {len(inconsistencias)} registros")
print("\n💡 Dataset está pronto para análise exploratória!")
print("="*80)

In [ ]:
# ============================================================================
# CÉLULA 8: CRIAÇÃO DE DATASETS & ESTATÍSTICAS DESCRITIVAS
# ============================================================================
# Descrição: Cria versão agregada + estatísticas ampliadas
# Estratégia: df_agregado (produção) + df_original (preços)
# ============================================================================

print("="*80)
print("📊 CRIAÇÃO DE DATASETS ESTRATÉGICOS")
print("="*80)

# --- 1. PRESERVAR DATASET ORIGINAL ---
df_original = df.copy()
print(f"\n✅ Dataset Original preservado: {df_original.shape[0]} linhas × {df_original.shape[1]} colunas")
print("   💡 USO: Análise de variação e volatilidade de PREÇOS")

# --- 2. CRIAR DATASET AGREGADO ---
print("\n📦 Criando dataset agregado (município + ano + produto)...")

df_agregado = df.groupby(['municipio', 'ano', 'produto'], as_index=False).agg({
    'area_colhida_ha': 'first',  # Valor único (igual em todos os registros duplicados)
    'quantidade_produzida_ton': 'first',  # Valor único
    'rendimento_medio_kg_ha': 'first',  # Valor único
    'preco_medio_anual_r$_kg': 'mean',  # MÉDIA dos preços (agregação real)
    'valor_producao_r$': 'mean'  # MÉDIA dos valores
})

print(f"✅ Dataset Agregado criado: {df_agregado.shape[0]} linhas × {df_agregado.shape[1]} colunas")
print("   💡 USO: Análises de PRODUÇÃO, ÁREA, RENDIMENTO, GEOGRAFIA e TEMPO")

# Verificar estrutura
print(f"\n🔍 Verificação:")
print(f"   - Municípios únicos: {df_agregado['municipio'].nunique()}")
print(f"   - Anos únicos: {df_agregado['ano'].nunique()} (de {df_agregado['ano'].min()} a {df_agregado['ano'].max()})")
print(f"   - Produtos únicos: {df_agregado['produto'].nunique()}")
print(f"   - Combinações esperadas: {df_agregado['municipio'].nunique()} × {df_agregado['ano'].nunique()} × {df_agregado['produto'].nunique()} = {df_agregado['municipio'].nunique() * df_agregado['ano'].nunique() * df_agregado['produto'].nunique()}")
print(f"   - Combinações reais: {df_agregado.shape[0]}")

if df_agregado.shape[0] == df_agregado['municipio'].nunique() * df_agregado['ano'].nunique() * df_agregado['produto'].nunique():
    print("   ✅ Dataset balanceado (todas as combinações presentes)")
else:
    print("   ⚠️ Há algumas combinações ausentes (normal se algum município não produziu determinado produto em algum ano)")

# --- 3. ADICIONAR COLUNAS DERIVADAS (ENGENHARIA DE FEATURES) ---
print("\n" + "="*80)
print("🔧 ENGENHARIA DE FEATURES (COLUNAS DERIVADAS)")
print("="*80)

# 3.1. Receita por Hectare (R$/ha)
df_agregado['receita_por_ha'] = df_agregado['valor_producao_r$'] / df_agregado['area_colhida_ha']
print("✅ Criada: receita_por_ha (R$/ha)")

# 3.2. Taxa de Crescimento Anual (será calculada por grupo depois)
# Por enquanto, apenas ordenar para facilitar cálculos posteriores
df_agregado = df_agregado.sort_values(['municipio', 'produto', 'ano']).reset_index(drop=True)
print("✅ Dataset ordenado por município → produto → ano")

# 3.3. Década (para análises por período)
df_agregado['decada'] = (df_agregado['ano'] // 10) * 10
print("✅ Criada: decada (2010, 2020)")

print(f"\n📊 Novas dimensões do dataset agregado: {df_agregado.shape[0]} linhas × {df_agregado.shape[1]} colunas")

# --- 4. ESTATÍSTICAS DESCRITIVAS AVANÇADAS ---
print("\n" + "="*80)
print("📈 ESTATÍSTICAS DESCRITIVAS AVANÇADAS (DATASET AGREGADO)")
print("="*80)

def calcular_estatisticas_avancadas(series, nome_coluna):
    """
    Calcula estatísticas avançadas para uma série numérica.

    Retorna dicionário com:
    - Tendência central: média, mediana, média geométrica
    - Dispersão: desvio padrão, CV, IQR
    - Forma: skewness, kurtosis
    - Extremos: min, max, range
    """
    return {
        'Coluna': nome_coluna,
        'Count': len(series),
        'Mean': series.mean(),
        'Median': series.median(),
        'Std': series.std(),
        'CV (%)': (series.std() / series.mean() * 100) if series.mean() != 0 else np.nan,
        'Min': series.min(),
        'Q25': series.quantile(0.25),
        'Q75': series.quantile(0.75),
        'Max': series.max(),
        'IQR': series.quantile(0.75) - series.quantile(0.25),
        'Range': series.max() - series.min(),
        'Skewness': series.skew(),
        'Kurtosis': series.kurtosis()
    }

# Colunas numéricas (exceto ano e década)
colunas_numericas = [
    'area_colhida_ha',
    'quantidade_produzida_ton',
    'rendimento_medio_kg_ha',
    'preco_medio_anual_r$_kg',
    'valor_producao_r$',
    'receita_por_ha'
]

# Calcular estatísticas para cada coluna
stats_list = []
for col in colunas_numericas:
    stats = calcular_estatisticas_avancadas(df_agregado[col], col)
    stats_list.append(stats)

# Criar DataFrame de estatísticas
df_stats = pd.DataFrame(stats_list)

# Formatar para exibição
df_stats_display = df_stats.copy()
for col in ['Mean', 'Median', 'Std', 'Min', 'Q25', 'Q75', 'Max', 'IQR', 'Range']:
    df_stats_display[col] = df_stats_display[col].apply(lambda x: f"{x:,.2f}" if pd.notnull(x) else "-")
df_stats_display['CV (%)'] = df_stats_display['CV (%)'].apply(lambda x: f"{x:.2f}%" if pd.notnull(x) else "-")
df_stats_display['Skewness'] = df_stats_display['Skewness'].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "-")
df_stats_display['Kurtosis'] = df_stats_display['Kurtosis'].apply(lambda x: f"{x:.3f}" if pd.notnull(x) else "-")

print("\n📊 Estatísticas Completas:")
display(df_stats_display.T)

# --- 5. INTERPRETAÇÕES ---
print("\n" + "="*80)
print("💡 INTERPRETAÇÕES ESTATÍSTICAS")
print("="*80)

for idx, row in df_stats.iterrows():
    col_name = row['Coluna']
    cv = row['CV (%)']
    skew = row['Skewness']
    kurt = row['Kurtosis']

    print(f"\n🔍 {col_name.replace('_', ' ').title()}:")

    # Interpretação de CV (Coeficiente de Variação)
    if cv < 15:
        print(f"   - Dispersão: BAIXA (CV={cv:.1f}%) → Dados homogêneos")
    elif cv < 30:
        print(f"   - Dispersão: MODERADA (CV={cv:.1f}%) → Variabilidade aceitável")
    else:
        print(f"   - Dispersão: ALTA (CV={cv:.1f}%) → Dados heterogêneos")

    # Interpretação de Skewness (Assimetria)
    if abs(skew) < 0.5:
        print(f"   - Assimetria: SIMÉTRICA (Skew={skew:.2f}) → Distribuição equilibrada")
    elif skew > 0.5:
        print(f"   - Assimetria: POSITIVA (Skew={skew:.2f}) → Cauda à direita (valores altos)")
    else:
        print(f"   - Assimetria: NEGATIVA (Skew={skew:.2f}) → Cauda à esquerda (valores baixos)")

    # Interpretação de Kurtosis (Curtose)
    if abs(kurt) < 1:
        print(f"   - Curtose: MESOCÚRTICA (Kurt={kurt:.2f}) → Distribuição normal")
    elif kurt > 1:
        print(f"   - Curtose: LEPTOCÚRTICA (Kurt={kurt:.2f}) → Pico alto, caudas pesadas (outliers)")
    else:
        print(f"   - Curtose: PLATICÚRTICA (Kurt={kurt:.2f}) → Pico baixo, distribuição achatada")

# --- 6. SALVAR DATASETS ---
print("\n" + "="*80)
print("💾 DATASETS FINALIZADOS")
print("="*80)
print(f"✅ df_original: {df_original.shape[0]} linhas (para análise de preços)")
print(f"✅ df_agregado: {df_agregado.shape[0]} linhas (para análises principais)")
print("\n💡 Próximo: Análise Temporal, Geográfica e Correlações!")
print("="*80)


In [ ]:
# ============================================================================
# CÉLULA 9: ANÁLISE TEMPORAL
# ============================================================================
# Descrição: Análise de tendências, crescimento e padrões temporais
# Dataset: df_agregado (48 linhas)
# ============================================================================

print("="*80)
print("📈 ANÁLISE TEMPORAL — EVOLUÇÃO DA PRODUÇÃO (2013-2024)")
print("="*80)

# --- 1. PREPARAÇÃO DOS DADOS TEMPORAIS ---
# Criar agregações por ano e produto (somando municípios)
df_temporal = df_agregado.groupby(['ano', 'produto'], as_index=False).agg({
    'area_colhida_ha': 'sum',
    'quantidade_produzida_ton': 'sum',
    'rendimento_medio_kg_ha': 'mean',  # Média ponderada seria ideal, mas média simples é aceitável
    'preco_medio_anual_r$_kg': 'mean',
    'valor_producao_r$': 'sum'
})

print(f"✅ Dados temporais agregados: {df_temporal.shape[0]} observações")

# --- 2. GRÁFICOS DE EVOLUÇÃO TEMPORAL ---

# 2.1. Evolução da Produção (Toneladas)
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Subplot 1: Produção Total por Produto
ax1 = axes[0, 0]
for i, produto in enumerate(['Uva', 'Manga']):
    dados = df_temporal[df_temporal['produto'] == produto]
    ax1.plot(dados['ano'], dados['quantidade_produzida_ton'],
             marker='o', linewidth=2.5, markersize=8,
             color=COLORS_LINE[i], label=produto)

ax1.set_title('Evolução da Produção Total (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Ano', fontweight='bold')
ax1.set_ylabel('Produção (toneladas)', fontweight='bold')
ax1.legend(loc='best', frameon=True)
ax1.grid(axis='y', alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_large_numbers(x)))

# Anotar crescimento total
for i, produto in enumerate(['Uva', 'Manga']):
    dados = df_temporal[df_temporal['produto'] == produto]
    valor_inicial = dados[dados['ano'] == 2013]['quantidade_produzida_ton'].values[0]
    valor_final = dados[dados['ano'] == 2024]['quantidade_produzida_ton'].values[0]
    crescimento = ((valor_final / valor_inicial) - 1) * 100

    ax1.text(0.05, 0.95 - (i * 0.08),
             f'{produto}: +{crescimento:.1f}% (12 anos)',
             transform=ax1.transAxes, fontsize=10,
             bbox=dict(boxstyle='round', facecolor=COLORS_LINE[i], alpha=0.3))

# Subplot 2: Área Colhida (Hectares)
ax2 = axes[0, 1]
for i, produto in enumerate(['Uva', 'Manga']):
    dados = df_temporal[df_temporal['produto'] == produto]
    ax2.plot(dados['ano'], dados['area_colhida_ha'],
             marker='s', linewidth=2.5, markersize=8,
             color=COLORS_LINE[i], label=produto)

ax2.set_title('Evolução da Área Colhida (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Ano', fontweight='bold')
ax2.set_ylabel('Área Colhida (hectares)', fontweight='bold')
ax2.legend(loc='best', frameon=True)
ax2.grid(axis='y', alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_large_numbers(x)))

# Subplot 3: Rendimento Médio (Kg/ha)
ax3 = axes[1, 0]
for i, produto in enumerate(['Uva', 'Manga']):
    dados = df_temporal[df_temporal['produto'] == produto]
    ax3.plot(dados['ano'], dados['rendimento_medio_kg_ha'],
             marker='^', linewidth=2.5, markersize=8,
             color=COLORS_LINE[i], label=produto)

ax3.set_title('Evolução do Rendimento Médio (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax3.set_xlabel('Ano', fontweight='bold')
ax3.set_ylabel('Rendimento (kg/hectare)', fontweight='bold')
ax3.legend(loc='best', frameon=True)
ax3.grid(axis='y', alpha=0.3)
ax3.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_large_numbers(x)))

# Subplot 4: Valor da Produção (R$)
ax4 = axes[1, 1]
for i, produto in enumerate(['Uva', 'Manga']):
    dados = df_temporal[df_temporal['produto'] == produto]
    ax4.plot(dados['ano'], dados['valor_producao_r$'],
             marker='D', linewidth=2.5, markersize=8,
             color=COLORS_LINE[i], label=produto)

ax4.set_title('Evolução do Valor da Produção (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax4.set_xlabel('Ano', fontweight='bold')
ax4.set_ylabel('Valor da Produção (R$)', fontweight='bold')
ax4.legend(loc='best', frameon=True)
ax4.grid(axis='y', alpha=0.3)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'R$ {x/1e9:.1f}B' if x >= 1e9 else f'R$ {x/1e6:.0f}M'))

plt.suptitle('Painel de Análise Temporal — Vale do São Francisco (2013-2024)',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# --- 3. CÁLCULO DE TAXAS DE CRESCIMENTO (CAGR) ---
print("\n" + "="*80)
print("📊 TAXAS DE CRESCIMENTO ANUAL COMPOSTO (CAGR)")
print("="*80)

cagr_results = []

for produto in ['Uva', 'Manga']:
    dados_produto = df_temporal[df_temporal['produto'] == produto].sort_values('ano')

    # CAGR de Produção
    prod_2013 = dados_produto[dados_produto['ano'] == 2013]['quantidade_produzida_ton'].values[0]
    prod_2024 = dados_produto[dados_produto['ano'] == 2024]['quantidade_produzida_ton'].values[0]
    cagr_prod = calcular_cagr(prod_2013, prod_2024, 11)

    # CAGR de Área
    area_2013 = dados_produto[dados_produto['ano'] == 2013]['area_colhida_ha'].values[0]
    area_2024 = dados_produto[dados_produto['ano'] == 2024]['area_colhida_ha'].values[0]
    cagr_area = calcular_cagr(area_2013, area_2024, 11)

    # CAGR de Valor
    valor_2013 = dados_produto[dados_produto['ano'] == 2013]['valor_producao_r$'].values[0]
    valor_2024 = dados_produto[dados_produto['ano'] == 2024]['valor_producao_r$'].values[0]
    cagr_valor = calcular_cagr(valor_2013, valor_2024, 11)

    cagr_results.append({
        'Produto': produto,
        'CAGR Produção (%)': f"{cagr_prod:.2f}%",
        'CAGR Área (%)': f"{cagr_area:.2f}%",
        'CAGR Valor (%)': f"{cagr_valor:.2f}%",
        'Crescimento Total Produção': f" +{((prod_2024/prod_2013 - 1)*100):.1f}%"
    })

df_cagr = pd.DataFrame(cagr_results)
display(df_cagr)

# --- 4. ANÁLISE DE ACELERAÇÃO/DESACELERAÇÃO ---
print("\n" + "="*80)
print("🚀 ANÁLISE DE ACELERAÇÃO/DESACELERAÇÃO")
print("="*80)

# Calcular variação percentual ano a ano
for produto in ['Uva', 'Manga']:
    print(f"\n📊 {produto}:")
    dados_produto = df_temporal[df_temporal['produto'] == produto].sort_values('ano').copy()
    dados_produto['var_producao_%'] = dados_produto['quantidade_produzida_ton'].pct_change() * 100

    # Identificar períodos de aceleração/desaceleração
    for idx, row in dados_produto[dados_produto['ano'] >= 2014].iterrows():
        var = row['var_producao_%']
        ano = int(row['ano'])
        if pd.notna(var):
            if var > 10:
                print(f"   🚀 {ano-1}→{ano}: +{var:.1f}% (CRESCIMENTO FORTE)")
            elif var > 0:
                print(f"   ↗️  {ano-1}→{ano}: +{var:.1f}% (crescimento moderado)") # Fixed f-string
            elif var > -10:
                print(f"   ↘️  {ano-1}→{ano}: {var:.1f}% (queda moderada)")
            else:
                print(f"   ⚠️  {ano-1}→{ano}: {var:.1f}% (QUEDA FORTE)")

# --- 5. INSIGHTS TEMPORAIS ---
print("\n" + "="*80)
print("💡 PRINCIPAIS INSIGHTS TEMPORAIS")
print("="*80)

# Produto com maior crescimento
produto_maior_crescimento = df_cagr.loc[df_cagr['CAGR Produção (%)'].str.replace('%', '').astype(float).idxmax(), 'Produto']
print(f"\n✅ Produto com maior CAGR: {produto_maior_crescimento}")

# Anos de pico
for produto in ['Uva', 'Manga']:
    dados_produto = df_temporal[df_temporal['produto'] == produto]
    ano_pico_prod = dados_produto.loc[dados_produto['quantidade_produzida_ton'].idxmax(), 'ano']
    ano_pico_valor = dados_produto.loc[dados_produto['valor_producao_r$'].idxmax(), 'ano']
    print(f"\n📊 {produto}:")
    print(f"   - Ano de pico de produção: {int(ano_pico_prod)}")
    print(f"   - Ano de pico de valor: {int(ano_pico_valor)}")

print("\n" + "="*80)
print("✅ ANÁLISE TEMPORAL CONCLUÍDA!")
print("="*80)

In [ ]:
# ============================================================================
# CÉLULA 10: ANÁLISE GEOGRÁFICA
# ============================================================================
# Descrição: Compara Juazeiro (BA) vs. Petrolina (PE)
# Dataset: df_agregado (48 linhas)
# ============================================================================

print("="*80)
print("🗺️ ANÁLISE GEOGRÁFICA — JUAZEIRO (BA) vs. PETROLINA (PE)")
print("="*80)

# --- 1. AGREGAÇÃO POR MUNICÍPIO ---
df_municipal = df_agregado.groupby('municipio', as_index=False).agg({
    'area_colhida_ha': 'sum',
    'quantidade_produzida_ton': 'sum',
    'rendimento_medio_kg_ha': 'mean',
    'valor_producao_r$': 'sum',
    'receita_por_ha': 'mean'
})

print("\n📊 Totais por Município (2013-2024):")
display(df_municipal)

# Calcular % de participação
df_municipal['% Produção'] = (df_municipal['quantidade_produzida_ton'] / df_municipal['quantidade_produzida_ton'].sum() * 100).round(2)
df_municipal['% Valor'] = (df_municipal['valor_producao_r$'] / df_municipal['valor_producao_r$'].sum() * 100).round(2)

print("\n🔍 Participação Relativa:")
for _, row in df_municipal.iterrows():
    print(f"   {row['municipio']}: {row['% Produção']:.1f}% da produção | {row['% Valor']:.1f}% do valor")

# --- 2. GRÁFICOS COMPARATIVOS ---
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Subplot 1: Produção Total por Município
ax1 = axes[0, 0]
municipios = df_municipal['municipio'].values
producao = df_municipal['quantidade_produzida_ton'].values
colors_municipios = [CUSTOM_PALETTE['primary'], CUSTOM_PALETTE['secondary']]

bars1 = ax1.barh(municipios, producao, color=colors_municipios, edgecolor='black', linewidth=1)
ax1.set_title('Produção Total Acumulada (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Produção (toneladas)', fontweight='bold')
ax1.grid(axis='x', alpha=0.3)

# Adicionar valores nas barras
for bar, valor in zip(bars1, producao):
    width = bar.get_width()
    ax1.text(width * 0.5, bar.get_y() + bar.get_height()/2,
             f'{valor:,.0f} ton\n({valor/producao.sum()*100:.1f}%)',
             ha='center', va='center', fontsize=11, fontweight='bold', color='white')

# Subplot 2: Valor da Produção por Município
ax2 = axes[0, 1]
valor = df_municipal['valor_producao_r$'].values / 1e9  # Converter para bilhões

bars2 = ax2.barh(municipios, valor, color=colors_municipios, edgecolor='black', linewidth=1)
ax2.set_title('Valor da Produção Acumulado (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Valor (R$ Bilhões)', fontweight='bold')
ax2.grid(axis='x', alpha=0.3)

for bar, val in zip(bars2, valor):
    width = bar.get_width()
    ax2.text(width * 0.5, bar.get_y() + bar.get_height()/2,
             f'R$ {val:.1f}B\n({val/(valor.sum())*100:.1f}%)',
             ha='center', va='center', fontsize=11, fontweight='bold', color='white')

# Subplot 3: Composição por Produto (Petrolina)
ax3 = axes[1, 0]
petrolina_produtos = df_agregado[df_agregado['municipio'] == 'Petrolina (PE)'].groupby('produto')['quantidade_produzida_ton'].sum()
colors_produtos = [COLORS_LINE[0] if prod == 'Uva' else COLORS_LINE[1] for prod in petrolina_produtos.index]

wedges, texts, autotexts = ax3.pie(
    petrolina_produtos.values,
    labels=petrolina_produtos.index,
    autopct='%1.1f%%',
    colors=colors_produtos,
    startangle=90,
    explode=(0.05, 0.05),
    textprops={'fontsize': 11, 'fontweight': 'bold'}
)
ax3.set_title('Composição da Produção — Petrolina (PE)', fontsize=14, fontweight='bold', pad=15)

# Subplot 4: Composição por Produto (Juazeiro)
ax4 = axes[1, 1]
juazeiro_produtos = df_agregado[df_agregado['municipio'] == 'Juazeiro (BA)'].groupby('produto')['quantidade_produzida_ton'].sum()
colors_produtos_j = [COLORS_LINE[0] if prod == 'Uva' else COLORS_LINE[1] for prod in juazeiro_produtos.index]

wedges, texts, autotexts = ax4.pie(
    juazeiro_produtos.values,
    labels=juazeiro_produtos.index,
    autopct='%1.1f%%',
    colors=colors_produtos_j,
    startangle=90,
    explode=(0.05, 0.05),
    textprops={'fontsize': 11, 'fontweight': 'bold'}
)
ax4.set_title('Composição da Produção — Juazeiro (BA)', fontsize=14, fontweight='bold', pad=15)

plt.suptitle('Análise Geográfica — Comparação entre Municípios',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

# --- 3. ANÁLISE DE ESPECIALIZAÇÃO POR MUNICÍPIO-PRODUTO ---
print("\n" + "="*80)
print("🏆 RANKING POR MUNICÍPIO E PRODUTO")
print("="*80)

df_ranking = df_agregado.groupby(['municipio', 'produto'], as_index=False).agg({
    'quantidade_produzida_ton': 'sum',
    'area_colhida_ha': 'sum',
    'rendimento_medio_kg_ha': 'mean',
    'valor_producao_r$': 'sum'
})

df_ranking = df_ranking.sort_values('quantidade_produzida_ton', ascending=False)
display(df_ranking)

# Identificar especialização
print("\n💡 Especialização por Município:")
for municipio in ['Petrolina (PE)', 'Juazeiro (BA)']:
    dados_mun = df_ranking[df_ranking['municipio'] == municipio]
    produto_dominante = dados_mun.iloc[0]['produto']
    producao_dominante = dados_mun.iloc[0]['quantidade_produzida_ton']
    producao_total_mun = dados_mun['quantidade_produzida_ton'].sum()
    pct_dominante = (producao_dominante / producao_total_mun) * 100

    print(f"\n   📍 {municipio}:")
    print(f"      - Produto dominante: {produto_dominante} ({pct_dominante:.1f}%)")
    if pct_dominante > 70:
        print(f"      - Classificação: ALTA ESPECIALIZAÇÃO")
    elif pct_dominante > 50:
        print(f"      - Classificação: ESPECIALIZAÇÃO MODERADA")
    else:
        print(f"      - Classificação: PRODUÇÃO DIVERSIFICADA")

# --- 4. EVOLUÇÃO COMPARATIVA (LINHA DO TEMPO) ---
print("\n" + "="*80)
print("📈 EVOLUÇÃO COMPARATIVA POR MUNICÍPIO")
print("="*80)

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Gráfico 1: Produção ao longo dos anos
ax1 = axes[0]
for i, municipio in enumerate(['Petrolina (PE)', 'Juazeiro (BA)']):
    dados = df_agregado[df_agregado['municipio'] == municipio].groupby('ano')['quantidade_produzida_ton'].sum()
    ax1.plot(dados.index, dados.values, marker='o', linewidth=2.5, markersize=7,
             color=colors_municipios[i], label=municipio)

ax1.set_title('Evolução da Produção por Município (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Ano', fontweight='bold')
ax1.set_ylabel('Produção (toneladas)', fontweight='bold')
ax1.legend(loc='best')
ax1.grid(axis='y', alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_large_numbers(x)))

# Gráfico 2: Rendimento médio ao longo dos anos
ax2 = axes[1]
for i, municipio in enumerate(['Petrolina (PE)', 'Juazeiro (BA)']):
    dados = df_agregado[df_agregado['municipio'] == municipio].groupby('ano')['rendimento_medio_kg_ha'].mean()
    ax2.plot(dados.index, dados.values, marker='s', linewidth=2.5, markersize=7,
             color=colors_municipios[i], label=municipio)

ax2.set_title('Evolução do Rendimento Médio por Município (2013-2024)', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Ano', fontweight='bold')
ax2.set_ylabel('Rendimento (kg/hectare)', fontweight='bold')
ax2.legend(loc='best')
ax2.grid(axis='y', alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: format_large_numbers(x)))

plt.tight_layout()
plt.show()

# --- 5. ÍNDICE DE CONCENTRAÇÃO (HERFINDAHL-HIRSCHMAN) ---
print("\n" + "="*80)
print("📊 ÍNDICE DE CONCENTRAÇÃO GEOGRÁFICA (HHI)")
print("="*80)

# Calcular market share de cada município
market_shares = df_municipal['quantidade_produzida_ton'] / df_municipal['quantidade_produzida_ton'].sum()
hhi = (market_shares ** 2).sum()

print(f"\n🔍 HHI (Herfindahl-Hirschman Index): {hhi:.4f}")
print("\n💡 Interpretação:")
if hhi > 0.25:
    print("   ⚠️ ALTA CONCENTRAÇÃO — Mercado dominado por um município")
    print("   Recomendação: Diversificar produção geograficamente para reduzir risco")
elif hhi > 0.15:
    print("   📊 CONCENTRAÇÃO MODERADA — Produção relativamente equilibrada")
else:
    print("   ✅ BAIXA CONCENTRAÇÃO — Produção bem distribuída")

print(f"\n   Participações:")
for _, row in df_municipal.iterrows():
    share = (row['quantidade_produzida_ton'] / df_municipal['quantidade_produzida_ton'].sum()) * 100
    print(f"      - {row['municipio']}: {share:.1f}%")

print("\n" + "="*80)
print("✅ ANÁLISE GEOGRÁFICA CONCLUÍDA!")
print("="*80)

In [ ]:
# ============================================================================
# CÉLULA 11: ANÁLISE DE CORRELAÇÕES E TESTES ESTATÍSTICOS
# ============================================================================
# Descrição: Correlações, testes de hipótese e feature importance
# Dataset: df_agregado (48 linhas)
# ============================================================================

print("="*80)
print("🔗 ANÁLISE DE CORRELAÇÕES E TESTES ESTATÍSTICOS")
print("="*80)

# --- 1. MATRIZ DE CORRELAÇÃO (PEARSON) ---
print("\n" + "="*80)
print("📊 MATRIZ DE CORRELAÇÃO (PEARSON)")
print("="*80)

# Selecionar apenas colunas numéricas (exceto ano e década)
colunas_correlacao = [
    'area_colhida_ha',
    'quantidade_produzida_ton',
    'rendimento_medio_kg_ha',
    'preco_medio_anual_r$_kg',
    'valor_producao_r$',
    'receita_por_ha'
]

# Calcular matriz de correlação
corr_matrix = df_agregado[colunas_correlacao].corr()

# Visualização em Heatmap
fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))  # Máscara triangular superior

sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt='.2f',
    cmap='RdYlGn',
    center=0,
    square=True,
    linewidths=1,
    cbar_kws={"shrink": 0.8, "label": "Correlação de Pearson"},
    vmin=-1,
    vmax=1,
    ax=ax
)

ax.set_title('Matriz de Correlação — Variáveis Numéricas', fontsize=14, fontweight='bold', pad=15)
plt.tight_layout()
plt.show()

# Identificar correlações fortes
print("\n💡 Correlações Fortes (|r| > 0.7):")
correlacoes_fortes = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        valor_corr = corr_matrix.iloc[i, j]
        if abs(valor_corr) > 0.7:
            var1 = corr_matrix.columns[i]
            var2 = corr_matrix.columns[j]
            correlacoes_fortes.append({
                'Variável 1': var1,
                'Variável 2': var2,
                'Correlação': f"{valor_corr:.3f}",
                'Interpretação': 'Positiva Forte' if valor_corr > 0 else 'Negativa Forte'
            })
            print(f"   • {var1} ↔ {var2}: r = {valor_corr:.3f} ({'positiva' if valor_corr > 0 else 'negativa'})")

if not correlacoes_fortes:
    print("   Nenhuma correlação forte (|r| > 0.7) encontrada.")

# --- 2. TESTE DE NORMALIDADE (SHAPIRO-WILK) ---
print("\n" + "="*80)
print("🧪 TESTE DE NORMALIDADE (SHAPIRO-WILK)")
print("="*80)
print("H₀: A distribuição é normal (p > 0.05)")
print("H₁: A distribuição NÃO é normal (p ≤ 0.05)")

normalidade_results = []
for col in colunas_correlacao:
    stat, p_value = shapiro(df_agregado[col].dropna())
    is_normal = p_value > 0.05
    normalidade_results.append({
        'Variável': col,
        'Estatística': f"{stat:.4f}",
        'p-value': f"{p_value:.4f}",
        'Distribuição': 'Normal ✅' if is_normal else 'Não-Normal ⚠️'
    })

df_normalidade = pd.DataFrame(normalidade_results)
display(df_normalidade)

# --- 3. TESTE T / MANN-WHITNEY (UVA vs. MANGA) ---
print("\n" + "="*80)
print("🧪 TESTE DE DIFERENÇA ENTRE PRODUTOS (UVA vs. MANGA)")
print("="*80)

# Testar diferença de rendimento entre Uva e Manga
rendimento_uva = df_agregado[df_agregado['produto'] == 'Uva']['rendimento_medio_kg_ha']
rendimento_manga = df_agregado[df_agregado['produto'] == 'Manga']['rendimento_medio_kg_ha']

# Verificar normalidade
normal_uva = shapiro(rendimento_uva)[1] > 0.05
normal_manga = shapiro(rendimento_manga)[1] > 0.05

print(f"\n🔍 Testando: Rendimento Médio (kg/ha)")
print(f"   - Uva: Média = {rendimento_uva.mean():.0f} kg/ha (n={len(rendimento_uva)})")
print(f"   - Manga: Média = {rendimento_manga.mean():.0f} kg/ha (n={len(rendimento_manga)})")

# Escolher teste apropriado
if normal_uva and normal_manga:
    # Teste t (paramétrico)
    stat, p_value = ttest_ind(rendimento_uva, rendimento_manga)
    test_name = "Teste t de Student"
    print(f"\n   ✅ Ambas distribuições normais → Usando {test_name}")
else:
    # Mann-Whitney U (não-paramétrico)
    stat, p_value = mannwhitneyu(rendimento_uva, rendimento_manga, alternative='two-sided')
    test_name = "Mann-Whitney U"
    print(f"\n   ⚠️ Pelo menos uma distribuição não-normal → Usando {test_name}")

print(f"\n📊 Resultados do {test_name}:")
print(f"   - Estatística: {stat:.4f}")
print(f"   - p-value: {p_value:.4f}")
print(f"\n💡 Conclusão (α = 0.05):")
if p_value < 0.05:
    print(f"   ✅ Há DIFERENÇA SIGNIFICATIVA entre os rendimentos (p={p_value:.4f} < 0.05)")
    diferenca_pct = ((rendimento_uva.mean() / rendimento_manga.mean() - 1) * 100)
    if diferenca_pct > 0:
        print(f"   → Uva tem rendimento {abs(diferenca_pct):.1f}% SUPERIOR à Manga")
    else:
        print(f"   → Manga tem rendimento {abs(diferenca_pct):.1f}% SUPERIOR à Uva")
else:
    print(f"   ⚠️ NÃO há diferença significativa entre os rendimentos (p={p_value:.4f} > 0.05)")

# --- 4. ANOVA (COMPARAÇÃO ENTRE MUNICÍPIOS) ---
print("\n" + "="*80)
print("🧪 ANOVA — COMPARAÇÃO ENTRE MUNICÍPIOS")
print("="*80)

# Testar diferença de produção entre municípios
juazeiro_prod = df_agregado[df_agregado['municipio'] == 'Juazeiro (BA)']['quantidade_produzida_ton']
petrolina_prod = df_agregado[df_agregado['municipio'] == 'Petrolina (PE)']['quantidade_produzida_ton']

stat_anova, p_value_anova = f_oneway(juazeiro_prod, petrolina_prod)

print(f"\n🔍 Testando: Quantidade Produzida (ton) entre municípios")
print(f"   - Juazeiro (BA): Média = {juazeiro_prod.mean():.0f} ton/ano")
print(f"   - Petrolina (PE): Média = {petrolina_prod.mean():.0f} ton/ano")

print(f"\n📊 Resultados da ANOVA:")
print(f"   - Estatística F: {stat_anova:.4f}")
print(f"   - p-value: {p_value_anova:.4f}")
print(f"\n💡 Conclusão (α = 0.05):")
if p_value_anova < 0.05:
    print(f"   ✅ Há DIFERENÇA SIGNIFICATIVA entre os municípios (p={p_value_anova:.4f} < 0.05)")
    print(f"   → Petrolina produz significativamente mais que Juazeiro")
else:
    print(f"   ⚠️ NÃO há diferença significativa entre os municípios (p={p_value_anova:.4f} > 0.05)")

# --- 5. CORRELAÇÃO DE SPEARMAN (RANKINGS) ---
print("\n" + "="*80)
print("📊 CORRELAÇÃO DE SPEARMAN (RANKINGS)")
print("="*80)
print("💡 Útil para relações não-lineares ou ordinais")

# Correlação entre área e produção
corr_spearman, p_spearman = spearmanr(df_agregado['area_colhida_ha'], df_agregado['quantidade_produzida_ton'])

print(f"\n🔍 Testando: Área Colhida ↔ Quantidade Produzida")
print(f"   - Correlação de Spearman (ρ): {corr_spearman:.4f}")
print(f"   - p-value: {p_spearman:.4f}")
print(f"\n💡 Interpretação:")
if abs(corr_spearman) > 0.9:
    print(f"   ✅ Correlação MUITO FORTE (ρ={corr_spearman:.3f})")
elif abs(corr_spearman) > 0.7:
    print(f"   ✅ Correlação FORTE (ρ={corr_spearman:.3f})")
elif abs(corr_spearman) > 0.5:
    print(f"   📊 Correlação MODERADA (ρ={corr_spearman:.3f})")
else:
    print(f"   ⚠️ Correlação FRACA (ρ={corr_spearman:.3f})")

# --- 6. FEATURE IMPORTANCE (RANDOM FOREST) ---
print("\n" + "="*80)
print("🌳 FEATURE IMPORTANCE — RANDOM FOREST")
print("="*80)
print("💡 Identifica variáveis mais importantes para prever Valor de Produção")

# Preparar dados
X = df_agregado[['area_colhida_ha', 'quantidade_produzida_ton', 'rendimento_medio_kg_ha', 'preco_medio_anual_r$_kg']]
y = df_agregado['valor_producao_r$']

# Normalizar features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Treinar Random Forest
rf_model = RandomForestRegressor(n_estimators=100, random_state=42, max_depth=5)
rf_model.fit(X_scaled, y)

# Obter importâncias
feature_importance = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf_model.feature_importances_
}).sort_values('Importance', ascending=False)

print("\n📊 Ranking de Importância das Variáveis:")
display(feature_importance)

# Visualizar
fig, ax = plt.subplots(figsize=(10, 5))
ax.barh(feature_importance['Feature'], feature_importance['Importance'], color=CUSTOM_PALETTE['primary'])
ax.set_xlabel('Importância', fontweight='bold')
ax.set_title('Feature Importance — Predição do Valor de Produção (Random Forest)', fontsize=14, fontweight='bold', pad=15)
ax.grid(axis='x', alpha=0.3)

for i, (idx, row) in enumerate(feature_importance.iterrows()):
    ax.text(row['Importance'] + 0.01, i, f"{row['Importance']:.3f}", va='center', fontsize=10)

plt.tight_layout()
plt.show()

print(f"\n💡 Variável mais importante: {feature_importance.iloc[0]['Feature']}")
print(f"   → Explica {feature_importance.iloc[0]['Importance']*100:.1f}% da variância do Valor de Produção")

print("\n" + "="*80)
print("✅ ANÁLISE DE CORRELAÇÕES E TESTES ESTATÍSTICOS CONCLUÍDA!")
print("="*80)

In [ ]:
# ============================================================================
# CÉLULA 10: CONCLUSÕES E RECOMENDAÇÕES ESTRATÉGICAS
# ============================================================================
# Descrição: Síntese dos principais insights e recomendações para stakeholders
# ============================================================================

print("="*80)
print("🎯 CONCLUSÕES E RECOMENDAÇÕES ESTRATÉGICAS")
print("="*80)

# --- 1. SÍNTESE DOS PRINCIPAIS INSIGHTS ---
print("\n" + "="*80)
print("📊 SÍNTESE DOS PRINCIPAIS INSIGHTS")
print("="*80)

insights = [
    {
        "Categoria": "Crescimento",
        "Insight": "Uva apresentou crescimento explosivo de +252% em 12 anos (CAGR 12.1% a.a.)",
        "Evidência": "Produção saltou de ~60K ton (2013) para ~210K ton (2024)",
        "Impacto": "🔥 CRÍTICO"
    },
    {
        "Categoria": "Concentração Geográfica",
        "Insight": "Petrolina domina 68.7% da produção e 79.1% do valor total",
        "Evidência": "HHI = 0.57 (alta concentração); Juazeiro representa apenas 31.3%",
        "Impacto": "⚠️ ALTO RISCO"
    },
    {
        "Categoria": "Especialização",
        "Insight": "Municípios têm especializações complementares: Petrolina (Uva 53%) vs. Juazeiro (Manga 84%)",
        "Evidência": "Juazeiro é ALTAMENTE especializado em Manga; Petrolina tem especialização moderada",
        "Impacto": "✅ OPORTUNIDADE"
    },
    {
        "Categoria": "Produtividade",
        "Insight": "Uva tem rendimento 29.5% SUPERIOR à Manga (diferença estatisticamente significativa, p=0.006)",
        "Evidência": "Uva: 38K kg/ha vs. Manga: 29K kg/ha (teste Mann-Whitney)",
        "Impacto": "✅ VALIDADO"
    },
    {
        "Categoria": "Volatilidade",
        "Insight": "Ambos produtos apresentaram quedas fortes em anos específicos (Uva: 2020/2022; Manga: 2014)",
        "Evidência": "Variações anuais de até -44.5% (Manga 2013→2014) e -21% (Uva 2019→2020)",
        "Impacto": "⚠️ RISCO CLIMÁTICO/MERCADO"
    },
    {
        "Categoria": "Valor Econômico",
        "Insight": "Valor de produção cresceu 252% (Uva) e 82% (Manga), impulsionado por preços (CAGR 21.6% e 13.1%)",
        "Evidência": "Preços médios subiram consistentemente; receita/ha tem correlação r=0.91 com preços",
        "Impacto": "💰 ALTA RENTABILIDADE"
    },
    {
        "Categoria": "Drivers de Valor",
        "Insight": "Quantidade Produzida explica 54.4% da variância do Valor de Produção (Feature Importance)",
        "Evidência": "Rendimento (27%) e Preço (14%) são fatores secundários; Área tem baixo impacto (5%)",
        "Impacto": "📈 ESCALA IMPORTA"
    }
]

df_insights = pd.DataFrame(insights)
display(df_insights)

# --- 2. RECOMENDAÇÕES ESTRATÉGICAS ---
print("\n" + "="*80)
print("💡 RECOMENDAÇÕES ESTRATÉGICAS")
print("="*80)

recomendacoes = """
### 🎯 **PARA GESTORES PÚBLICOS (Secretarias de Agricultura)**

1. **MITIGAR RISCO DE CONCENTRAÇÃO GEOGRÁFICA**
   - 🚨 Prioridade: ALTA
   - Problema: HHI = 0.57 (Petrolina domina 68.7%)
   - Ação: Incentivar expansão em Juazeiro e outros municípios do Vale (Casa Nova, Lagoa Grande)
   - Instrumentos:
     • Linhas de crédito subsidiadas (PRONAF, ABC+)
     • Assistência técnica especializada (Embrapa/IPA)
     • Investimento em infraestrutura de irrigação
   - Meta: Reduzir HHI para < 0.40 em 5 anos

2. **FORTALECER CADEIA PRODUTIVA DA MANGA EM JUAZEIRO**
   - 🎯 Prioridade: MÉDIA
   - Oportunidade: Juazeiro é altamente especializado (84%) mas tem menor rendimento
   - Ação: Programa de melhoramento genético e manejo integrado de pragas (MIP)
   - Meta: Aumentar rendimento de 29K para 35K kg/ha (+20%) até 2027
   - ROI Estimado: +R$ 300 milhões/ano em valor de produção

3. **CRIAR FUNDO ANTICÍCLICO PARA VOLATILIDADE**
   - ⚠️ Prioridade: MÉDIA
   - Problema: Quedas de até -44.5% em anos específicos
   - Ação: Seguro rural subsidiado + estoque regulador de preços mínimos
   - Parceiros: MAPA, Conab, seguradoras privadas

---

### 💼 **PARA PRODUTORES E ASSOCIAÇÕES**

4. **DIVERSIFICAR PORTFÓLIO DE VARIEDADES (UVA)**
   - 🍇 Prioridade: ALTA
   - Oportunidade: Uva tem CAGR de 12.1% (mercado aquecido)
   - Ação:
     • Introduzir variedades premium (Niágara Rosada, Red Globe)
     • Investir em certificações (GlobalGAP, Orgânico, Rainforest Alliance)
   - Mercado-alvo: Europa (85% das exportações), EUA, Ásia
   - ROI: Prêmio de 30-50% em preços certificados

5. **INTEGRAÇÃO VERTICAL (BENEFICIAMENTO)**
   - 💰 Prioridade: MÉDIA
   - Problema: 54.4% do valor vem de quantidade (escala)
   - Oportunidade: Agregar valor via processamento (sucos, vinhos, geleias, manga desidratada)
   - Ação: Cooperativas regionais para agroindústria
   - Exemplo: Cooperativa Aurora (SC) - ROI de 15% a.a.

6. **ADOÇÃO DE AGRICULTURA DE PRECISÃO**
   - 🛰️ Prioridade: MÉDIA-BAIXA
   - Justificativa: Rendimento explica 27% do valor (2º fator mais importante)
   - Ação: Sensoriamento remoto, fertirrigação automatizada, drones
   - Parceiros: Startups AgTech, Embrapa Digital
   - Meta: Reduzir custos em 15% e aumentar rendimento em 10%

---

### 🏦 **PARA INVESTIDORES E BANCOS**

7. **FINANCIAR EXPANSÃO EM MUNICÍPIOS PERIFÉRICOS**
   - 💵 Prioridade: ALTA
   - Oportunidade: Reduzir concentração + atender demanda crescente
   - Modelo: Fundos de investimento agrícola com participação minoritária (20-30%)
   - Retorno esperado: 18-22% a.a. (baseado em CAGR histórico de Uva)

8. **CRIAR LINHA DE CRÉDITO PARA IRRIGAÇÃO TECNIFICADA**
   - 💧 Prioridade: ALTA
   - Justificativa: Rendimento é 2º fator de valor (27%)
   - Produtos: Gotejamento de alta eficiência, pivôs centrais modernos
   - Condições: Prazo 10 anos, carência 2 anos, taxa TJLP + 2%

---

### 🔬 **PARA PESQUISADORES E UNIVERSIDADES**

9. **ESTUDOS DE RESILIÊNCIA CLIMÁTICA**
   - 🌡️ Prioridade: ALTA
   - Problema: Quedas abruptas em anos específicos (possivelmente climáticas)
   - Ação: Modelagem de risco climático + desenvolvimento de cultivares resistentes
   - Parceiros: Embrapa Semiárido, INSA, Univasf

10. **ANÁLISE DE PREÇOS E SAZONALIDADE**
    - 📊 Prioridade: MÉDIA
    - Oportunidade: Preços explicam 14% do valor (3º fator)
    - Ação: Estudo de séries temporais mensais (não disponível neste dataset anual)
    - Objetivo: Identificar janelas de comercialização ótimas (premium pricing)
"""

print(recomendacoes)

# --- 3. LIMITAÇÕES E PRÓXIMOS PASSOS ---
print("\n" + "="*80)
print("⚠️ LIMITAÇÕES DESTE ESTUDO")
print("="*80)

limitacoes = """
1. **GRANULARIDADE TEMPORAL**
   - Dados anuais não capturam sazonalidade intra-anual
   - Não é possível analisar timing de safras e picos de preço
   - Recomendação: Integrar dados mensais (se disponíveis)

2. **AUSÊNCIA DE VARIÁVEIS CLIMÁTICAS**
   - Não há dados de precipitação, temperatura, eventos extremos
   - Impossível correlacionar quedas produtivas com choques climáticos
   - Recomendação: Integrar dados INMET/ANA

3. **DADOS AGREGADOS POR MUNICÍPIO**
   - Não há informações sobre perfil de produtor (pequeno, médio, grande)
   - Não há dados de propriedades individuais
   - Recomendação: Estudo de caso com amostra de produtores

4. **PREÇOS MÚLTIPLOS POR ANO-PRODUTO**
   - Dataset original tem múltiplos registros de preço (possivelmente safras/variedades)
   - Agregamos para média anual (perda de detalhamento)
   - Recomendação: Investigar fonte original (IBGE PAM) para entender origem das duplicatas

5. **FALTA DE DADOS DE CUSTOS**
   - Apenas receitas (valor de produção)
   - Não é possível calcular margem de lucro ou rentabilidade líquida
   - Recomendação: Integrar com Cepea/Esalq (custos de produção)
"""

print(limitacoes)

# --- 4. PRÓXIMOS PASSOS (PESQUISA FUTURA) ---
print("\n" + "="*80)
print("🚀 PRÓXIMOS PASSOS PARA PESQUISA FUTURA")
print("="*80)

proximos_passos = """
1. **MODELAGEM PREDITIVA (MACHINE LEARNING)**
   - Desenvolver modelo de previsão de produção (SARIMA, Prophet, LSTM)
   - Objetivo: Antecipar safras e auxiliar planejamento logístico
   - Features: Clima, histórico, índices de vegetação (NDVI)

2. **ANÁLISE DE SÉRIES TEMPORAIS AVANÇADA**
   - Decomposição STL (tendência, sazonalidade, ruído)
   - Testes de causalidade de Granger (preços → produção?)
   - Modelos VAR (Vector Autoregression) para interação entre produtos

3. **DASHBOARD INTERATIVO (STREAMLIT/POWER BI)**
   - Permitir filtros dinâmicos (município, produto, período)
   - Visualizações interativas (zoom, hover tooltips)
   - Atualização automática com novos dados IBGE

4. **ANÁLISE ESPACIAL (GEOPROCESSAMENTO)**
   - Mapas coropléticos de produtividade por município
   - Análise de clustering espacial (Moran's I)
   - Identificação de "hotspots" de crescimento

5. **ESTUDO DE BENCHMARKING INTERNACIONAL**
   - Comparar Vale do São Francisco com outras regiões produtoras
     • Espanha (Múrcia), Chile (Vale do Elqui), África do Sul (Western Cape)
   - Identificar melhores práticas e gaps tecnológicos
"""

print(proximos_passos)

# --- 5. AGRADECIMENTOS E CONTATO ---
print("\n" + "="*80)
print("📬 CONTATO E COLABORAÇÃO")
print("="*80)

contato = f"""
Este notebook foi desenvolvido por **Everton Santos de Oliveira** como parte do portfólio de Data Science.

**📧 E-mail:** eso.datalab@gmail.com
**💼 LinkedIn:** https://www.linkedin.com/in/evert0n-sant0s/
**🐙 GitHub:** https://github.com/everton754
**📝 Medium:** https://medium.com/@eso.datalab
**🌐 Portfólio:** https://kenzie-portfolio-everton754.vercel.app/

---

### 🤝 **Interessado em discutir este projeto ou colaborar em análises de agronegócio?**

Este projeto demonstra competências em:
✅ Análise Exploratória de Dados (EDA)
✅ Visualização Profissional (Matplotlib/Seaborn)
✅ Testes Estatísticos (Shapiro-Wilk, Mann-Whitney, ANOVA, Spearman)
✅ Machine Learning (Random Forest Feature Importance)
✅ Storytelling com Dados
✅ Recomendações Estratégicas Acionáveis

**Entre em contato para discutir oportunidades em:**
- Ciência de Dados para Agronegócio
- Análise de Séries Temporais
- Dashboards Analíticos (Power BI, Streamlit)
- Modelagem Preditiva (Machine Learning)

---

**Outros Projetos no Portfólio:**
🌾 Previsão de Preços de Commodities Agrícolas
📊 Dashboard de Monitoramento de Safras (Streamlit)
🌧️ Impacto de Variáveis Climáticas na Produtividade

---

*Última atualização: 13 de novembro de 2025*
*Dataset: IBGE PAM (Produção Agrícola Municipal) 2013-2024*
*Ferramenta: Python 3.11+ • Jupyter/Google Colab*
"""

print(contato)

print("\n" + "="*80)
print("✅ ANÁLISE EXPLORATÓRIA DE DADOS CONCLUÍDA COM SUCESSO!")
print("="*80)
print("\n🎉 Parabéns! Notebook profissional finalizado e pronto para portfólio!")
print("💡 Próximos passos: Exportar como HTML/PDF e publicar no GitHub + LinkedIn")
print("="*80)